In [56]:
import math

cost_matrix = [
    [999,   5, 4,   3,   999, 999, 999], #nodo 1
    [999, 999, 999, 2,   3,   999, 11],  #nodo 2
    [999, 999, 999, 1,   999, 4,   10],  #...
    [999, 999, 999, 999, 5,   6,   9],
    [999, 999, 999, 999, 999, 999, 4],
    [999, 999, 999, 999, 999, 999, 3],
    [999, 999, 999, 999, 999, 999, 999]
]

def best_route(cost_matrix):
    l = len(cost_matrix)
    routes = [ ['']*l for i in ['']*l ]
    costs = [ [0]*l for i in [0]* l ]

    for node in range(l): #nodo
        for dest in range(l): #destino
            m = cost_matrix[node][dest]
            routes[node][dest] = node

            for ndest in range(node, dest):
                if costs[node][ndest] + cost_matrix[ndest][dest] < m:
                    m = costs[node][ndest] + cost_matrix[ndest][dest]
                    routes[node][dest] = ndest

                costs[node][dest] = m

    return routes, costs


def calc_route(routes, start, end):
    if start == end:
        return start
    else:
        return '%s, %s' % (calc_route(routes, start, routes[start][end]), end)


routes, costs = best_route(cost_matrix)
calc_route(routes, 0, 6)

'0, 2, 5, 6'